In [132]:
#import pytraj as pt
#import nglview as nv

import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import scipy as sp
from scipy import stats

#import bokeh as bk
#import ggplot
#from ggplot import *

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

import os
import sys
import gc
import copy

import tqdm
import itertools

In [ ]:
baseDir='.'

If you are viewing this in google colab, you will need to clone the repository first.
To do so uncomment the two code cells below

In [ ]:
#!git clone https://github.com/wesleymsmith/Piezo_PIP2_binding_analysis.git

In [ ]:
#baseDir=Piezo_PIP2_binding_analysis.git

In [10]:
xcelData=pd.read_excel(baseDir+'Residue_ID_total_occupancy_10_1_2019.xlsx',
              sheet_name=None)

In [11]:
@interact
def show_data(sheet_name=xcelData.keys()):
    return xcelData[sheet_name]

aW50ZXJhY3RpdmUoY2hpbGRyZW49KERyb3Bkb3duKGRlc2NyaXB0aW9uPXUnc2hlZXRfbmFtZScsIG9wdGlvbnM9KHUnY2cnLCB1J1NoZWV0MScsIHUnYWEnLCB1J3RlbnNpb25fMzBucycsIHXigKY=


In [18]:
#the above tables are
#aa - result summary for all atom simulation
#cg - result summary for coarse grain simulation
#tension_30ns - results of all atom simulation with membrane tension
#sheet 1 is apparently blank...
#resinfo_table - mapping between cryo-em structure sequence and all atom residue ids
resinfoDataSheet=xcelData['resinfo_table']
resinfoTable=resinfoDataSheet[
    resinfoDataSheet.columns[[0,3,5,7]]][2:]
resinfoTable.columns=['PDB_ID','Arm1_Resid','Arm2_Resid','Arm3_Resid']
resinfoTable.head()

,PDB_ID,Arm1_Resid,Arm2_Resid,Arm3_Resid
2,782,1,1419,2837
3,783,2,1420,2838
4,784,3,1421,2839
5,785,4,1422,2840
6,786,5,1423,2841


In [28]:
#the pdb residue id's are sequential, but have gaps corresponding
#to unresolved amino acids in the Cryo-EM structure.
#The easy solution is just to fill in linearly.
#The three arms in our simulation structure are identical, so
#we can generate our back-map by just repeating the pdb sequence 3 times
simResid_to_pdbResid=map(int,list(np.arange(
    np.array(resinfoTable['PDB_ID'])[0],
    782+len(resinfoTable['PDB_ID'])))*3)
', '.join(map(str,simResid_to_pdbResid))

'782, 783, 784, 785, 786, 787, 788, 789, 790, 791, 792, 793, 794, 795, 796, 797, 798, 799, 800, 801, 802, 803, 804, 805, 806, 807, 808, 809, 810, 811, 812, 813, 814, 815, 816, 817, 818, 819, 820, 821, 822, 823, 824, 825, 826, 827, 828, 829, 830, 831, 832, 833, 834, 835, 836, 837, 838, 839, 840, 841, 842, 843, 844, 845, 846, 847, 848, 849, 850, 851, 852, 853, 854, 855, 856, 857, 858, 859, 860, 861, 862, 863, 864, 865, 866, 867, 868, 869, 870, 871, 872, 873, 874, 875, 876, 877, 878, 879, 880, 881, 882, 883, 884, 885, 886, 887, 888, 889, 890, 891, 892, 893, 894, 895, 896, 897, 898, 899, 900, 901, 902, 903, 904, 905, 906, 907, 908, 909, 910, 911, 912, 913, 914, 915, 916, 917, 918, 919, 920, 921, 922, 923, 924, 925, 926, 927, 928, 929, 930, 931, 932, 933, 934, 935, 936, 937, 938, 939, 940, 941, 942, 943, 944, 945, 946, 947, 948, 949, 950, 951, 952, 953, 954, 955, 956, 957, 958, 959, 960, 961, 962, 963, 964, 965, 966, 967, 968, 969, 970, 971, 972, 973, 974, 975, 976, 977, 978, 979, 980, 981,

While the above excel sheet provides a useful summary at a glance, we would like to have direct access to the distribution of residence times rather than just the mean max and cummulative sum over all lipids. 

Using the set of all individually measured residence times, we can fit a model distribution. More specifically, the reciporical of residence time would correspond to a frequency. Specifically, the reciporical of residence time gives us the corresponding unbinding frequency. 

This can then be used to fit an appropriate distribution (geometric distribution would be one choice) and provide a characteristic unbinding frequency (or characteristic residence time as its reciporical). More over, it can give us a bound / confidence interval of this distribution as well.

We can then repeat this process for the all atom model. While CG is expected to have shorter residence times due to the notably lower membrane viscocity, we should still be able to see if the ranking and / or relative characteristic residence time / unbinding frequencies match (for each protein amino acid). If CG can rank amino acids in the correct order, based upon PIP2 residence times, then we can be confident that it is functioning well as a model for correctly predicting lipid binding sites.

Below, this residence time distribution data can be exctracted from the coarse grain simulation data files which list individual PIP2 residence time observations for each protein residue (amino acid)

in the Raw_PIP2_CG_residence_time_data directory, the .xvg files contain
the 'occupancy' of each amino acid at each output time step.
The occupancy is zero if there were no PIP2 lipids in contact
and non-zero if there was at least one PIP2 lipid in contact.
The first step is to extract these individual timeseries
into a joint table.

In [16]:
#os.listdir can be used to generate a list of all xvg files present
cg_RawData_dir=baseDir+'Raw_PIP2_CG_residence_time_data/'
cg_dataFile_list=[dataFileName for dataFileName in os.listdir(cg_RawData_dir) \
             if 'xvg' in dataFileName]
cg_dataFile_list

['id_4236_mask.xvg',
 'id_3365_mask.xvg',
 'id_2250_mask.xvg',
 'id_3630_mask.xvg',
 'id_3456_mask.xvg',
 'id_843_mask.xvg',
 'id_3516_mask.xvg',
 'id_548_mask.xvg',
 'id_186_mask.xvg',
 'id_3043_mask.xvg',
 'id_1988_mask.xvg',
 'id_3679_mask.xvg',
 'id_4110_mask.xvg',
 'id_2218_mask.xvg',
 'id_373_mask.xvg',
 'id_1483_mask.xvg',
 'id_1813_mask.xvg',
 'id_3034_mask.xvg',
 'id_1354_mask.xvg',
 'id_2857_mask.xvg',
 'id_162_mask.xvg',
 'id_3794_mask.xvg',
 'id_577_mask.xvg',
 'id_244_mask.xvg',
 'id_4026_mask.xvg',
 'id_245_mask.xvg',
 'id_3529_mask.xvg',
 'id_2772_mask.xvg',
 'id_782_mask.xvg',
 'id_2521_mask.xvg',
 'id_1046_mask.xvg',
 'id_2388_mask.xvg',
 'id_3415_mask.xvg',
 'id_1983_mask.xvg',
 'id_378_mask.xvg',
 'id_2212_mask.xvg',
 'id_1580_mask.xvg',
 'id_4013_mask.xvg',
 'id_800_mask.xvg',
 'id_788_mask.xvg',
 'id_475_mask.xvg',
 'id_613_mask.xvg',
 'id_2325_mask.xvg',
 'id_3523_mask.xvg',
 'id_1642_mask.xvg',
 'id_680_mask.xvg',
 'id_2416_mask.xvg',
 'id_3350_mask.xvg',
 'id_17

In [34]:
#Each of the above files contains an 18 line header, followed by 2 column occupancy series
#the data can be easily extracted using pd.read_table
cgDataTables=[]
for dataFileName in tqdm.tqdm_notebook(cg_dataFile_list):
    resID=int(dataFileName.split('_')[1])
    seqID=simResid_to_pdbResid[resID-1]
    tempTable=pd.read_csv(
        cg_RawData_dir+dataFileName,
        skiprows=17,names=['Time','Occupancy'],
        delim_whitespace=True)
    tempTable['ResID']=resID
    tempTable['SeqID']=seqID
    tempTable['Frame']=np.arange(len(tempTable))
    tempTable=tempTable[['ResID','SeqID','Frame','Time','Occupancy']]
    cgDataTables.append(copy.deepcopy(tempTable))
    
cg_occupancy_data=pd.concat(cgDataTables)
cg_occupancy_data.head()

,ResID,SeqID,Frame,Time,Occupancy
0,4236,2181,0,0.0,0
1,4236,2181,1,1000.0,0
2,4236,2181,2,2000.0,0
3,4236,2181,3,3000.0,0
4,4236,2181,4,4000.0,0


Now we can extract residence times from this occupancy by first finding
all 'runs' within the occupancy series of each residue.

This can be accomplished using itertools.groupby to obtain the lengths of each continguous interval where occupancy was non-zero. Since these are discrete integer values, we can easily use the function 'unique' to bin them into a histogram like form by setting the 'return_counts' option to 'True'

We can then plot the resulting distribution...
It seems to look quite exponential like, so it would make sense to try and fit either a geometric (if we think of each frame like an individual 'trial') or an exponential (if want to think of each run's length as a 'wait time')

The exponential distribution with a mean (or characteristic length, $\lambda$) is given by:
$$p(x,\lambda)=\frac{e^{-x/\lambda}}{\lambda}$$


To fit an exponential distribution to this data we first compute the characteristic length ($\gamma$) as the mean ($\bar{X}$) of the observed residence times ($X_i$)

The variance of an exponential distribution with a characteristic length (mean) of $\lambda$ is $\lambda^2$.

Thus for a given confidence level $\gamma$ we may construct the interval:

$$ \lambda \in (\bar{X}-Z_\gamma\sqrt{\frac{\bar{X}^2}{n}},\bar{X}+Z_\gamma\sqrt{\frac{\bar{X}^2}{n}}) $$

Below, we show the estimation procedure for an arbitrarily selected residue.

In [ ]:
def extract_runs(x):
    return [len(list(g)) for k,g in itertools.groupby(x, bool) if k]

In [131]:
@interact
def fit_exp_dist(tempResID=cg_occupancy_data[cg_occupancy_data.Occupancy>0].ResID.unique()):
    tempDat=cg_occupancy_data[cg_occupancy_data['ResID']==tempResID]
    print "First five entries of occupancy data for resid %g"%tempResID
    print tempDat.Occupancy.head()
    print '---'
    print "first five observed 'run' lengths in occupancy"
    runLengths=extract_runs(np.array(tempDat.Occupancy))
    print runLengths[:5]
    print '---'
    print "Histogram of run lengths"
    residenceTimeDist=np.unique(runLengths,return_counts=True)
    print residenceTimeDist
    plt.scatter(residenceTimeDist[0],residenceTimeDist[1]/(1.*(np.sum(residenceTimeDist[1]))))
    #plt.show()

    #lets get a 95% confidence interval... the needed coefficient is found as sp.stats.norm.ppf(q=.975)
    #n, the number of observations, is equal to the sum of our histogram counts
    lambdaEst=np.sum(residenceTimeDist[0]*residenceTimeDist[1])/np.sum(residenceTimeDist[1])*1.
    lambdaCIradius=sp.stats.norm.ppf(q=.975)*np.sqrt(1.0*lambdaEst**2/np.sum(residenceTimeDist[1]))

    print "Most likely estimate: lambda = %.3f"%lambdaEst
    print "At 95%% confidence: %.3f <= lambda <= %.3f"%(lambdaEst-lambdaCIradius,
                                    lambdaEst+lambdaCIradius)
    lval=lambdaEst-lambdaCIradius
    plt.plot(residenceTimeDist[0],map(lambda x: np.exp(-x/(lval))/lval,residenceTimeDist[0]),
             '#aa0000')
    lval=lambdaEst
    plt.plot(residenceTimeDist[0],map(lambda x: np.exp(-x/(lval))/lval,residenceTimeDist[0]),
             'r')
    lval=lambdaEst+lambdaCIradius
    plt.plot(residenceTimeDist[0],map(lambda x: np.exp(-x/(lval))/lval,residenceTimeDist[0]),
             '#aa0000')
    plt.show()

aW50ZXJhY3RpdmUoY2hpbGRyZW49KERyb3Bkb3duKGRlc2NyaXB0aW9uPXUndGVtcFJlc0lEJywgb3B0aW9ucz0oMzM2NSwgMjI1MCwgODQzLCAxNDgzLCAzMDM0LCAxNjIsIDI0NCwgMjQ1LCDigKY=
